In [2]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.RealisticPendulum import RealisticPendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


In [3]:
# Load Data:
with jnp.load('Data/data.npz') as data:
    X_data1 = data['observation_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['observation_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)
print(X_data[:10,:], y_data[:10,:])
rng = jax.random.PRNGKey(0)

(1782000, 5) (1782000, 4)
[[-4.4167123  -3.4106708   0.3245155  -4.3028955  -2.85725   ]
 [-4.3906307  -3.5248356   0.75463617 -2.3850248   2.1163223 ]
 [-4.3518276  -3.5722234   1.0373462  -0.7954778   1.8793108 ]
 [-4.319883   -3.5952773   0.7226013  -0.49457547  0.08415771]
 [-4.3183722  -3.643409   -0.17949304 -1.4100746  -1.9696319 ]
 [-4.314437   -3.6466212   0.18593678  0.34977925  1.8161194 ]
 [-4.313418   -3.6165287  -0.02583007  0.88862765  0.07477856]
 [-4.3292494  -3.5834537  -0.514141    0.81375325 -0.9096937 ]
 [-4.354113   -3.5321815  -0.65593374  1.437035    0.0316143 ]
 [-4.393692   -3.4772806  -1.0963553   1.3554939  -0.97768736]] [[-4.3906307  -3.5248356   0.75463617 -2.3850248 ]
 [-4.3518276  -3.5722234   1.0373462  -0.7954778 ]
 [-4.319883   -3.5952773   0.7226013  -0.49457547]
 [-4.3183722  -3.643409   -0.17949304 -1.4100746 ]
 [-4.314437   -3.6466212   0.18593678  0.34977925]
 [-4.313418   -3.6165287  -0.02583007  0.88862765]
 [-4.3292494  -3.5834537  -0.514141  

In [4]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.0001, num_epochs=100, batch_size=64)

Epoch 0, Train Loss: 0.8081, Test Loss: 0.8124
Epoch 10, Train Loss: 0.4502, Test Loss: 0.4490
Epoch 20, Train Loss: 0.4122, Test Loss: 0.4114
Epoch 30, Train Loss: 0.4238, Test Loss: 0.4225
Epoch 40, Train Loss: 0.4156, Test Loss: 0.4152
Epoch 50, Train Loss: 0.4152, Test Loss: 0.4154
Epoch 60, Train Loss: 0.3943, Test Loss: 0.3950
Epoch 70, Train Loss: 0.3714, Test Loss: 0.3724
Epoch 80, Train Loss: 0.3762, Test Loss: 0.3779
Epoch 90, Train Loss: 0.3656, Test Loss: 0.3674


In [5]:
from env1.RealisticPendulum import RealisticPendulum as env1

env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
#print(State.pipeline_state)
#State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )
print(qqd10, qqd11)



11
[[-4.3792067]
 [-3.3238752]
 [ 1.5568799]
 [-1.4642515]] [-4.4399147 -3.260322   1.3311983 -2.1098776]


In [6]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
